In [1]:
from utils import *

In [2]:
import logging
from random import randint
import random
import errno
import functools
from loguru import logger
import signal
import time
import json
import faiss
import subprocess
import shutil
import psutil
import pickle
from collections import defaultdict
from datetime import datetime

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

import lean_dojo
from lean_dojo import *
from utils.lean_math_utils import *
from math import floor

print("Imports done")

fin = open('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/tac_template_freq.json', 'r')
tac_template_freq = json.load(fin)
fin.close()

fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
traced_repo = pickle.load(fin)
fin.close()

print("Tactics loaded")

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
index = faiss.read_index('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/faiss_index_bert_embeds-batch64-60k-loss0047.idx')
model_state = torch.load('/home/mcwave/code/automath/atp/datasets/rag_tactic_templates/bert_embeder_state-batch64-60k-loss0047.model')

MAX_MEMORY_USAGE = 16*1024*1024*1024
MAX_STEPS = 100000
MAX_TACTIC_FROM_TEMPLATE = 50
PENALTY_SEEN_TARGET_MULTIPLIER = 3
MAX_NUM_DOJO_ATTEMPT = 2
MAX_NUM_OUTPUT_PER_STATE = 50

Imports done
Tactics loaded


/home/mcwave/anaconda3/envs/atp/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/tmp/ipykernel_6607/2658190431.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be lo

In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForTextEncoding
from typing import Union, List

tokenizer_tacgen = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model_tacgen = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

# tokenizer_retrieval = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small")
# model_retrieval = AutoModelForTextEncoding.from_pretrained("kaiyuy/leandojo-lean4-retriever-byt5-small")

# tokenizer_retrieval_tacgen = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small")
# model_retrieval_tacgen = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-retriever-tacgen-byt5-small")

In [4]:
import os
from lean_dojo import *
import random

print("Imports done")

# fin = open('/home/mcwave/code/automath/atp/datasets/traced_repo_math_in_lean.pkl', 'rb')
# traced_repo = pickle.load(fin)
# fin.close()
# print("Loaded traced repo")

github_token = os.getenv("GITHUB_ACCESS_TOKEN", None)
# print("token:", github_token)
repo = LeanGitRepo(
    "https://github.com/leanprover-community/mathematics_in_lean",
    "210802794074c003ddf237ee56a0bf1fb5a9fe10"
#     "https://github.com/yangky11/miniF2F-lean4",
#     "6bcf0b4940fbf17a1ba83db4ed639fbcb26b1a27"
)

Imports done


In [5]:
# @torch.no_grad()
# def encode(s: Union[str, List[str]]) -> torch.Tensor:
#     """Encode texts into feature vectors."""
#     if isinstance(s, str):
#         s = [s]
#         should_squeeze = True
#     else:
#         should_squeeze = False
#     tokenized_s = tokenizer_retrieval(s, return_tensors="pt", padding=True)
#     hidden_state = model_retrieval(tokenized_s.input_ids).last_hidden_state
#     lens = tokenized_s.attention_mask.sum(dim=1)
#     features = (hidden_state * tokenized_s.attention_mask.unsqueeze(2)).sum(dim=1) / lens.unsqueeze(1)
#     if should_squeeze:
#         features = features.squeeze()
#     return features

# @torch.no_grad()
# def retrieve(state: str, premises: List[str], k: int) -> List[str]:
#     """Retrieve the top-k premises given a state."""
#     state_emb = encode(state)
#     premise_embs = encode(premises)
#     scores = (state_emb @ premise_embs.T)
#     topk = scores.topk(k).indices.tolist()
#     return [premises[i] for i in topk]

In [6]:
# def reprover_rag(state):
#     retrieved_premises = retrieve(state, all_tacs, k=4)
#     input = "\n\n".join(retrieved_premises + [state])
#     tokenized_input = tokenizer_retrieval_tacgen(input, return_tensors="pt", max_length=2300, truncation=True)

#     # Generate multiple tactics via beam search.
#     tactic_candidates_ids = model_retrieval_tacgen.generate(
#         tokenized_input.input_ids,
#         max_length=1024,
#         num_beams=4,
#         length_penalty=0.0,
#         do_sample=False,
#         num_return_sequences=4,
#         early_stopping=False,
#     )
#     tactic_candidates = tokenizer.batch_decode(
#         tactic_candidates_ids, skip_special_tokens=True
#     )
# #     print("candidates", tactic_candidates)
#     return tactic_candidates[0]

In [7]:
def reprover_normal(state):
    tokenized_state = tokenizer_tacgen(state, return_tensors="pt")
#     print(state, tokenized_state)
    # Generate a single tactic.
#     tactic_ids = model.generate(tokenized_state.input_ids, max_length=1024)
#     tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
#     print(tactic, end="\n\n")

    # Generate multiple tactics via beam search.
    tactic_candidates_ids = model_tacgen.generate(
        tokenized_state.input_ids,
        do_sample = True,
        max_length=1024
    )
#     print(tactic_candidates_ids)
    tactic_candidates = tokenizer_tacgen.batch_decode(
        tactic_candidates_ids, skip_special_tokens=True
    )
#     print("candidates", tactic_candidates)
    return tactic_candidates
state = """α : Type u_1
inst✝ : Lattice α
x y z : α
⊢ x ⊓ y = y ⊓ x
"""
reprover_normal(state)

['simp_rw [inf_comm, inf_eq_right.2]']

In [19]:
def get_proof_reprover(file_path, full_name):
    theorem = Theorem(repo, file_path, full_name)
    dojo, state_0 = Dojo(theorem).__enter__()
    curr_state = state_0
    pp_to_state = {}
    seen = set()
    beam = [(explore_state_complexity(curr_state.pp), curr_state.pp)]
    pp_to_state[curr_state.pp] = curr_state
    seen.add(curr_state.pp)
    beam_width = 2
    proven = False
    start_time = time.time()
    states_generated = []
#     print(beam)
    
    while beam:
#         print(2, beam)
        new_beam = []
        
        for _, state in beam[:beam_width]:
#             seen.add(state)
#             print("state", state)
#             print()
            state = pp_to_state[state]
#             if type(state) == lean_dojo.interaction.dojo.ProofFinished:
#                 return True, states_generated
            
            candidates = reprover_normal(state.pp)
#             print(candidates)
            for tac in candidates:
                if time.time() - start_time > 120:
                    return False, states_generated
                new_state = dojo.run_tac(state, tac)
#                 print("statetac", state, tac, new_state)
                if type(new_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
#                     print("Error")
                    continue
                elif type(new_state) == lean_dojo.interaction.dojo.ProofFinished:
#                     print("Tactic2:", tac)
#                     print("Found proof")
                    proven = True
                    continue
#                 print("Tactic:", state.pp, tac, new_state.pp)
                if new_state.pp in seen:
                    continue
#                 print("Adding state")
                states_generated.append(time.time() - start_time)
                complexity = explore_state_complexity(new_state.pp)
                pp_to_state[new_state.pp] = new_state
                new_beam.append((complexity, new_state.pp))
        if new_beam == []:
            pass
        else:
            beam = heapq.nsmallest(beam_width, new_beam)
        
#         if proven:
#             return True, states_generated
        
        if not beam:
            return False, states_generated
    
    return False, states_generated

def explore_state_complexity(state, base_complexity=None, seen_target_freq=None):
    if '⊢ False' in state:
        return 1000000
    if base_complexity is not None:
        return base_complexity + 1
    complexity = 0
    lines = state.split('\n')
    targets = []
    min_freq = 1000000
    for line in lines:
        if line.startswith("⊢"):
            target = line[1:].strip()
            targets.append(target)
            if seen_target_freq is not None:
                if target in seen_target_freq:
                    min_freq = min(min_freq, seen_target_freq[target])
                    seen_target_freq[target] = seen_target_freq[target] + 1
                else:
                    min_freq = 0
                    seen_target_freq[target] = 1
    lengths = [len(x) for x in targets]
    complexity = max(lengths)/2 + sum(lengths)/2
    if seen_target_freq is not None:
        complexity += PENALTY_SEEN_TARGET_MULTIPLIER*min_freq
    return complexity

In [9]:
def get_proof_reprover_nobeam(file_path, full_name):
    theorem = Theorem(repo, file_path, full_name)
    dojo, state_0 = Dojo(theorem).__enter__()
    curr_state = state_0
    proven = False
    start_time = time.time()
    seen = set()
    num_tries = 0
    
    signal.alarm(120)
    try:
#         while num_tries < 10:
        while True:
            if time.time() - start_time > 120:
                return False
            try:
                tac = reprover_normal(curr_state.pp)[0]
        #         print(curr_state.pp, tac, num_tries)
                new_state = dojo.run_tac(curr_state, tac)
        #         print(new_state)
        #         print("\n")
            except:
                num_tries += 1
                continue
            if type(new_state) in [LeanError,TimeoutError,TacticResult,DojoCrashError,DojoHardTimeoutError,DojoInitError,ProofGivenUp]:
                num_tries += 1
                continue
            elif type(new_state) == lean_dojo.interaction.dojo.ProofFinished:
                continue

            if new_state.pp in seen:
                continue
            seen.add(new_state.pp)
            curr_state = new_state
            num_tries = 0

        return False
    except TimeoutException:
        print("Out of time")
        return False
    finally:
        signal.alarm(0)

In [10]:
theorems_list = [("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_neg_cancel_right"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_left_cancel"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.add_right_cancel"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.zero_mul"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_eq_of_add_eq_zero"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.eq_neg_of_add_eq_zero"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_zero"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.neg_neg"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.self_sub"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.one_add_one_eq_two"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyRing.two_mul"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_right_inv"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_one"),
("MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean", "MyGroup.mul_inv_rev"),
("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb1"),
("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "absorb2"),
("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux1"),
("MIL/C02_Basics/S05_Proving_Facts_about_Algebraic_Structures.lean", "aux2"),
("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact1"),
("MIL/C02_Basics/S03_Using_Theorems_and_Lemmas.lean", "fact2"),
("MIL/C02_Basics/S04_More_on_Order_and_Divisibility.lean", "C02S04.aux"),
("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.le_abs_self"),
("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.neg_le_abs_self"),
("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_add"),
("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.lt_abs"),
("MIL/C03_Logic/S05_Disjunction.lean", "C03S05.MyAbs.abs_lt"),
("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.aux"),
("MIL/C03_Logic/S04_Conjunction_and_Iff.lean", "C03S04.not_monotone_iff"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.my_lemma4"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnUb"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnLb"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnEven"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.FnOdd"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.Subset.trans"),
("MIL/C03_Logic/S01_Implication_and_the_Universal_Quantifier.lean", "C03S01.SetUb"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.ConvergesTo"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_const"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_add"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul_const"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.exists_abs_le_of_convergesTo"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.aux"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_mul"),
("MIL/C03_Logic/S06_Sequences_and_Convergence.lean", "C03S06.convergesTo_unique"),
("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnUb"),
("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnLb"),
("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasUb"),
("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.FnHasLb"),
("MIL/C03_Logic/S02_The_Existential_Quantifier.lean", "C03S02.fnUb_add"),
("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnUb"),
("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnLb"),
("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasUb"),
("MIL/C03_Logic/S03_Negation.lean", "C03S03.FnHasLb"),
("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse"),
("MIL/C04_Sets_and_Functions/S02_Functions.lean", "inverse_spec"),
("MIL/C04_Sets_and_Functions/S02_Functions.lean", "Cantor"),
("MIL/C04_Sets_and_Functions/S01_Sets.lean", "primes"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbAux"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbSet"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sbFun"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_right_inv"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_injective"),
("MIL/C04_Sets_and_Functions/S03_The_Schroeder_Bernstein_Theorem.lean", "sb_surjective"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "fac"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "pow_two_le_fac"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "sum_sqr"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_add"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_add"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_comm"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.add_assoc"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_add"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.zero_mul"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.succ_mul"),
("MIL/C05_Elementary_Number_Theory/S02_Induction_and_Recursion.lean", "MyNat.mul_comm"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "Nat.Prime.eq_of_dvd_of_prime"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mem_of_dvd_prod_primes"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_infinite'"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.bounded_of_ex_finset"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.ex_finset_of_bounded"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.mod_4_eq_3_or_mod_4_eq_3"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.two_le_of_mod_4_eq_3"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.aux"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.exists_prime_factor_mod_4_eq_3"),
("MIL/C05_Elementary_Number_Theory/S03_Infinitely_Many_Primes.lean", "C05S03.primes_mod_4_eq_3_infinite"),
("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "even_of_even_sqr"),
("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_mul'"),
("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "factorization_pow'"),
("MIL/C05_Elementary_Number_Theory/S01_Irrational_Roots.lean", "Nat.Prime.factorization'"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.zero_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.one_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.add_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.neg_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mul_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.instCommRing"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.sub_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.div'_add_mod'"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.abs_mod'_le"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "Int.mod'_eq"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "aux"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "sq_add_sq_eq_zero"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_nonneg"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_eq_zero"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_pos"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mul"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_re"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.conj_im"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_conj"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.div_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.mod_def"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.norm_mod_lt"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.coe_natAbs_norm"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.natAbs_norm_mod_lt"),
("MIL/C06_Structures/S03_Building_the_Gaussian_Integers.lean", "gaussInt.not_norm_mul_left_lt_norm"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₁"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.add"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.neg"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.zero"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.Point.addGroupPoint"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.AddGroup₂"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasAddAddGroup₂"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasZeroAddGroup₂"),
("MIL/C06_Structures/S02_Algebraic_Structures.lean", "C06S02.hasNegAddGroup₂"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.add_assoc"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.Point.smul_distrib"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardTwoSimplex.weightedAverage"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.midpoint"),
("MIL/C06_Structures/S01_Structures.lean", "C06S01.StandardSimplex.weightedAverage"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid₁"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubMonoid₁Monoid"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubmonoidClass₁"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "Subgroup₁"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "SubgroupClass₁"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "Submonoid.Setoid"),
("MIL/C07_Hierarchies/S03_Subobjects.lean", "QuotientMonoid.mk"),
("MIL/C07_Hierarchies/S01_Basics.lean", "One₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "One₂"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Dia₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₂"),
("MIL/C07_Hierarchies/S01_Basics.lean", "DiaOneClass₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₂"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Inv₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Group₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_dia"),
("MIL/C07_Hierarchies/S01_Basics.lean", "dia_inv"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddSemigroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Semigroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Monoid₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "left_inv_eq_right_inv'"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommSemigroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "CommSemigroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommMonoid₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "CommMonoid₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddGroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "inv_eq_of_mul"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Group₃.mul_inv"),
("MIL/C07_Hierarchies/S01_Basics.lean", "mul_left_cancel₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "mul_right_cancel₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddCommGroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "CommGroup₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Ring₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "LE₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Preorder₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "PartialOrder₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "OrderedCommMonoid₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "SMul₃"),
("MIL/C07_Hierarchies/S01_Basics.lean", "Module₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "selfModule"),
("MIL/C07_Hierarchies/S01_Basics.lean", "nsmul₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "zsmul₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "abGrpModule"),
("MIL/C07_Hierarchies/S01_Basics.lean", "AddMonoid₄"),
("MIL/C07_Hierarchies/S01_Basics.lean", "mySMul"),
("MIL/C07_Hierarchies/S01_Basics.lean", "LT₁"),
("MIL/C07_Hierarchies/S01_Basics.lean", "PreOrder₂"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₁"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "isMonoidHom₂"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHom₁"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "AddMonoidHom₁"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "RingHom₁"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₁"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "badInst"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₂"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "map_inv_of_inv"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "MonoidHomClass₃"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHom"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresMonoidHom"),
("MIL/C07_Hierarchies/S02_Morphisms.lean", "OrderPresHomClass"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "eq_bot_iff_card"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "inf_bot_of_coprime"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "conjugate_one"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "aux_card_eq"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₁"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "iso₂"),
("MIL/C08_Groups_and_Rings/S01_Groups.lean", "finalIso"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_mk'"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_inj"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "isCoprime_Inf"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseMap_surj"),
("MIL/C08_Groups_and_Rings/S02_Rings.lean", "chineseIso"),
("MIL/C09_Topology/S03_Topological_Spaces.lean", "aux"),
("MIL/C09_Topology/S01_Filters.lean", "Tendsto₁"),
("MIL/C09_Topology/S02_Metric_Spaces.lean", """cauchySeq_of_le_geometric_two'""")]
# theorems_list = [('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean', 'MyRing.neg_add_cancel_left'),]

In [11]:
def get_thms(file_path):
    theorems_list = []
    f = open(file_path, "r")
    all_thms = f.read().split("theorem ")[1:]
    real_path = "/".join(file_path.split("/")[7:])
    for thm in all_thms:
        theorems_list.append((real_path, thm.split(" ")[0].replace("\n", "")))
    return theorems_list

In [12]:
# theorems_list = get_thms("/home/mcwave/code/automath/atp/miniF2F-lean4/MiniF2F/Test.lean") + get_thms("/home/mcwave/code/automath/atp/miniF2F-lean4/MiniF2F/Valid.lean")
# theorems_list

In [13]:
# # chapters = os.listdir("/home/mcwave/mathematics_in_lean/MIL")
# total_attempts = 0
# num_proven = 0
# # ReProver: 56-157, 5/15

# for thm_path, thm_name in theorems_list:
# #     try:
#     print(thm_path, thm_name)
#     is_proven, states_generated = get_proof_reprover_nobeam(thm_path, thm_name)
#     num_proven += is_proven
#     total_attempts += 1
#     state_data = {"proof_num": total_attempts, "is_proven": is_proven, "num_states": states_generated}
#     with open("/home/mcwave/code/automath/atp/datasets/reprover_results_nobeam.json", "a") as outfile:
#         json.dump(state_data, outfile)
#         outfile.write("\n")
#     print("res", num_proven, total_attempts)
# #     except Exception as e:
# #         print(e)
# print(1/0)

In [14]:
# fin = open('/home/mcwave/code/automath/atp/datasets/remaining_theorems_repo_mathlib4_20240617.pkl', 'rb')
# train_theorems = pickle.load(fin)
# fin.close()

# output_folder = '/home/mcwave/code/automath/atp/datasets/provability/rag/'
# #compute_provability_training_data_remote('.lake/packages/mathlib/Mathlib/Topology/Algebra/Module/Basic.lean', output_folder)

# all_file_paths = [] 
# for file_path, theorems in train_theorems.items():
#     if theorems is not None and len(theorems) > 0 and 'Mathlib' in file_path:
#         all_file_paths.append(file_path)
# random.shuffle(all_file_paths)

In [15]:
# total_proven = 0
# proof_idx = 0

# for file_path in all_file_paths[:1000]:
#     theorems = train_theorems[file_path]
#     for thm_name, val in theorems.items():
#         try:
#             is_proven, states_generated = get_proof_reprover_2(file_path, thm_name)
#             total_proven += is_proven
#             print("res", f"MIL/{chapter}/{sol[10:]}", thm_name, len(states_generated))
#             state_data = {"proof_idx": proof_idx, "is_proven": is_proven, "num_states": states_generated}
#             with open("/home/mcwave/code/automath/atp/datasets/reprover_results.json", "a") as outfile:
#                 json.dump(state_data, outfile)
#                 outfile.write("\n")
#         except Exception as e:
#             print(e)
#     proof_idx += 1
# print(1/0)

In [20]:
total_proven = 0
num_attempts = 0
total_states = 0

thms = []
start_idx = 0
#30/118 MIL

for thm_path, thm_name in theorems_list:
    try:
        print(thm_path, thm_name)
#         is_proven = get_proof_reprover_nobeam(thm_path, thm_name)
        is_proven, states_generated = get_proof_reprover(thm_path, thm_name)
#         total_proven += is_proven
        print(states_generated)
        total_states += len(states_generated)
        num_attempts += 1
        print(total_states, num_attempts)
        state_data = {"file_path": thm_path, "full_name": thm_name, "total_attempts": start_idx + num_attempts, "states_generated": len(states_generated)}
        with open("/home/mcwave/code/automath/atp/results/reprover/reprover_stategen_results_mil.json", "a") as outfile:
            json.dump(state_data, outfile)
            outfile.write("\n")
    except Exception as e:
        print(e)
print(1/0)

2024-10-16 21:35:13.148 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-16 21:35:13.150 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathematics_in_lean', commit='210802794074c003ddf237ee56a0bf1fb5a9fe10'), file_path=PosixPath('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean'), full_name='MyRing.add_neg_cancel_right')
2024-10-16 21:35:13.163 | INFO     | lean_dojo.interaction.dojo:__enter__:187 - Copy tree ...


MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean MyRing.add_neg_cancel_right
get_traced_repo_path, path= /home/mcwave/.cache/lean_dojo/leanprover-community-mathematics_in_lean-210802794074c003ddf237ee56a0bf1fb5a9fe10/mathematics_in_lean
The traced repo is available in the cache.


2024-10-16 21:35:14.986 | INFO     | lean_dojo.interaction.dojo:_modify_file:385 - Modifying MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean
2024-10-16 21:35:14.987 | INFO     | lean_dojo.interaction.dojo:_modify_file:407 - _modify_file: proof modified
2024-10-16 21:35:14.988 | INFO     | lean_dojo.interaction.dojo:_modify_file:425 - Creating Lean4Repl.lean at Lean4Repl.lean
2024-10-16 21:35:14.989 | INFO     | lean_dojo.interaction.dojo:_modify_file:434 - _modify_file: Done writing to Lean4Repl.lean
2024-10-16 21:35:15.548 | INFO     | lean_dojo.interaction.dojo:_modify_file:439 - _modify_file: All done. Writing to /tmp/tmp0vhzu670/mathematics_in_lean/MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean
2024-10-16 21:35:15.549 | INFO     | lean_dojo.interaction.dojo:__enter__:207 - lake build Lean4Repl ...
2024-10-16 21:35:15.550 | INFO     | lean_dojo.interaction.dojo:__enter__:210 - Launching the proof using <class 'lean_dojo.container.NativeContaine

Post processing ...
Returning ...


2024-10-16 21:37:53.935 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-16 21:37:53.935 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathematics_in_lean', commit='210802794074c003ddf237ee56a0bf1fb5a9fe10'), file_path=PosixPath('MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean'), full_name='MyRing.add_left_cancel')
2024-10-16 21:37:53.936 | INFO     | lean_dojo.interaction.dojo:__enter__:187 - Copy tree ...


...

2024-10-17 08:27:13.615 | INFO     | lean_dojo.interaction.dojo:_modify_file:385 - Modifying MIL/C09_Topology/S01_Filters.lean
2024-10-17 08:27:13.616 | WARNING  | lean_dojo.interaction.dojo:__init__:156 - Using Lean 4 without a hard timeout may hang indefinitely.
2024-10-17 08:27:13.617 | INFO     | lean_dojo.interaction.dojo:__enter__:168 - Initializing Dojo for Theorem(repo=LeanGitRepo(url='https://github.com/leanprover-community/mathematics_in_lean', commit='210802794074c003ddf237ee56a0bf1fb5a9fe10'), file_path=PosixPath('MIL/C09_Topology/S02_Metric_Spaces.lean'), full_name="cauchySeq_of_le_geometric_two'")
2024-10-17 08:27:13.618 | INFO     | lean_dojo.interaction.dojo:__enter__:187 - Copy tree ...


Failed to locate the theorem with `Tendsto₁` as its fully qualified name
MIL/C09_Topology/S02_Metric_Spaces.lean cauchySeq_of_le_geometric_two'
get_traced_repo_path, path= /home/mcwave/.cache/lean_dojo/leanprover-community-mathematics_in_lean-210802794074c003ddf237ee56a0bf1fb5a9fe10/mathematics_in_lean
The traced repo is available in the cache.


2024-10-17 08:27:15.666 | INFO     | lean_dojo.interaction.dojo:_modify_file:385 - Modifying MIL/C09_Topology/S02_Metric_Spaces.lean
2024-10-17 08:27:15.668 | INFO     | lean_dojo.interaction.dojo:_modify_file:407 - _modify_file: proof modified
2024-10-17 08:27:15.669 | INFO     | lean_dojo.interaction.dojo:_modify_file:425 - Creating Lean4Repl.lean at Lean4Repl.lean
2024-10-17 08:27:15.670 | INFO     | lean_dojo.interaction.dojo:_modify_file:434 - _modify_file: Done writing to Lean4Repl.lean
2024-10-17 08:27:15.670 | INFO     | lean_dojo.interaction.dojo:_modify_file:439 - _modify_file: All done. Writing to /tmp/tmp_51p5_0w/mathematics_in_lean/MIL/C09_Topology/S02_Metric_Spaces.lean
2024-10-17 08:27:15.671 | INFO     | lean_dojo.interaction.dojo:__enter__:207 - lake build Lean4Repl ...
2024-10-17 08:27:15.671 | INFO     | lean_dojo.interaction.dojo:__enter__:210 - Launching the proof using <class 'lean_dojo.container.NativeContainer'>
2024-10-17 08:27:15.671 | INFO     | lean_dojo.con

Post processing ...
Returning ...
[2.3158178329467773, 2.708635091781616, 3.101557731628418, 5.6067585945129395, 8.53809380531311, 12.754878759384155, 15.105027675628662, 18.125267505645752, 19.882081508636475, 20.293559789657593, 20.705113887786865, 22.02573823928833, 22.437565803527832, 24.000863075256348, 24.944518089294434, 27.074594736099243, 27.485995769500732, 27.897871494293213, 29.37041401863098, 31.99730110168457, 32.408830881118774, 34.9133083820343, 36.24562621116638, 36.65750598907471, 37.859031677246094, 39.36797618865967, 40.3124577999115, 40.723644733428955, 41.13522553443909, 46.385327100753784, 47.77034664154053, 50.60582494735718, 54.9799325466156, 57.499165773391724, 59.928850173950195, 61.24889326095581, 66.18121600151062, 66.59309768676758, 67.00455808639526, 70.49420857429504, 73.30922317504883, 73.7819242477417, 74.19397139549255, 74.6057755947113, 75.01721501350403, 76.22187781333923, 83.5040340423584, 85.74511981010437, 96.48533749580383, 104.36024904251099, 1

ZeroDivisionError: division by zero

In [ ]:
all_data_states = []
for thm in state_data:
    thm_num = thm["proof_num"]
    num_states = [0] + thm["num_states"]
    for i in range(12):
        if num_states[i+1] == 0:
            break
        all_data_states.append((120 * thm_num + 10 * i, num_states[i + 1] - num_states[i]))

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("kaiyuy/leandojo-lean4-tacgen-byt5-small")

# state = "n : ℕ\n⊢ gcd n n = n"
state = "G : Type u_1\ninst✝ : Group G\na : G\n⊢ a * a⁻¹ = 1"
tokenized_state = tokenizer(state, return_tensors="pt")

# Generate a single tactic.
tactic_ids = model.generate(tokenized_state.input_ids, max_length=1024)
tactic = tokenizer.decode(tactic_ids[0], skip_special_tokens=True)
print(tactic, end="\n\n")

# Generate multiple tactics via beam search.
tactic_candidates_ids = model.generate(
    tokenized_state.input_ids,
    max_length=1024,
    num_beams=4,
    length_penalty=0.0,
    do_sample=False,
    num_return_sequences=4,
    early_stopping=False,
)
tactic_candidates = tokenizer.batch_decode(
    tactic_candidates_ids, skip_special_tokens=True
)
for tac in tactic_candidates:
    print(tac)

In [ ]:
#Can theorem be proven?
#Number of tactics processed
#Number of distinct states
#Amount of time

In [ ]:
theorem = Theorem(repo, 'MIL/C02_Basics/S02_Proving_Identities_in_Algebraic_Structures.lean', "MyGroup.mul_right_inv") 

# For some theorems, it might take a few minutes.
dojo, state_0 = Dojo(theorem).__enter__()
print(state_0.pp)

# state_1 = dojo.run_tac(state_0, "simp")
# print(state_1.pp)

In [ ]:
theorem_code = "theorem mul_right_inv (a : G) : a * a⁻¹ = 1"
tactics = ["have h : (a * a⁻¹)⁻¹ * (a * a⁻¹ * (a * a⁻¹)) = 1 := by\nrw [mul_assoc, ← mul_assoc a⁻¹ a, mul_left_inv, one_mul, mul_left_inv]",
  "rw [← h, ← mul_assoc, mul_left_inv, one_mul]"]

print("Exploring states")
res = explore_states(dojo,
                   state_0,
                   theorem,
                   model_state,
                   tokenizer,
                   index, 
                   tacs,
                   theorem_code=theorem_code, 
                   proof_tactics=tactics,
                   max_steps=20000,
                   exit_on_finish=False,
                   verbose=False)

In [ ]:
vals[0]

In [ ]:
vals = list(res[0].values())
for i in vals:
    tacs = i[-1]
    # For some theorems, it might take a few minutes.
    test_state = state_0
    for tac in tacs:
        try:
            test_state = dojo.run_tac(test_state, tac)
        except:
            break
        if type(test_state) == lean_dojo.interaction.dojo.ProofFinished:
            print("Successful Proof")
            print("Tactics:", tacs)
            break

In [ ]:
res